# Workshop: Build a Multi-Agent System using Llama 3, Ollama, and CrewAI

**Objectives:**
- Understand the basics of LLM multi-agent systems
- Set up Ollama with Llama 3 and CrewAI
- Create a practical multi-agent use case: Business Report Assistant
- See how agents collaborate to solve complex tasks


## 1. What is a Multi-Agent LLM System?
- In GenAI, a *multi-agent system* is a pipeline of specialized LLM-powered agents, each with a distinct role and ability to interact.
- CrewAI is an open-source Python framework for orchestrating such LLM agents.
- Ollama lets you run powerful open-source LLMs (like Llama 3) locally and connect them to your apps.

## 2. Install Requirements
Run these in your terminal (not in notebook) if not already installed:
```bash
pip install crewai
pip install openai
```
**You must have [Ollama](https://ollama.com) running locally, with the Llama 3 model pulled:**
```bash
ollama pull llama3
ollama serve
```
- You can use a smaller model like `llama3:8b` if RAM is limited.

## 3. Use Case: Business Report Assistant
Suppose you want a system that, given a company name and sector:
1. **Research Agent:** Finds recent news and facts about the company.
2. **Analysis Agent:** Analyzes strengths and weaknesses from research.
3. **Writer Agent:** Summarizes findings into a readable business report.

We'll chain these agents using CrewAI, all powered by Llama 3.

## 4. Configure CrewAI to use Ollama Llama 3
CrewAI allows you to use local models via Ollama by setting the OpenAI-compatible endpoint.
Below, we set up a helper to call Llama 3 via Ollama.

In [1]:
!pip install crewai openai

  Using cached chromadb-1.0.13-cp39-abi3-macosx_11_0_arm64.whl.metadata (7.0 kB)
  Using cached json5-0.12.0-py3-none-any.whl.metadata (36 kB)
  Using cached onnxruntime-1.22.0-cp310-cp310-macosx_13_0_universal2.whl.metadata (4.5 kB)
  Using cached opentelemetry_api-1.34.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_sdk-1.34.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached pdfplumber-0.11.7-py3-none-any.whl.metadata (42 kB)
  Using cached tomli-2.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached tiktoken-0.9.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.10.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached cryptography-45.0.4-cp37-abi3-macosx_10_9_universal2.whl.metadata (5.7 kB)
  Using cached build-1.2

In [2]:
# If CrewAI or OpenAI aren't installed, uncomment below
# !pip install crewai openai
import os
os.environ["OPENAI_API_BASE"] = "http://localhost:11434/v1"
os.environ["OPENAI_API_KEY"] = "ollama"
# Now any OpenAI-compatible client, including CrewAI, will use Ollama Llama 3.

## 5. Define Agents and Their Roles
We'll define three agents:
- **ResearchAgent**: Gathers company information
- **AnalysisAgent**: Performs SWOT analysis
- **WriterAgent**: Writes a final report
Each agent gets its own system prompt.

In [1]:
from crewai import Agent, Task, Crew

research_agent = Agent(
    role='Research Agent',
    goal='Gather recent factual information about the company.',
    backstory='You are an expert business researcher, skilled at extracting recent news and facts from web snippets.',
    llm_model='llama3'
)
analysis_agent = Agent(
    role='Analysis Agent',
    goal="Analyze the company's strengths and weaknesses based on provided facts.",
    backstory='You are a management consultant specializing in SWOT analysis.',
    llm_model='llama3'
)
writer_agent = Agent(
    role='Writer Agent',
    goal='Summarize research and analysis into a clear business report.',
    backstory='You are a skilled business writer, creating executive-level summaries.',
    llm_model='llama3'
)

## 6. Define the Task Pipeline
- Each task can use the output of the previous agent.

In [2]:
# User input
company = 'Tesla'
sector = 'Electric Vehicles'

research_task = Task(
    description=f"Research recent news and facts about {company} in the {sector} sector.",
    agent=research_agent,
    expected_output="A concise bullet-point list of recent facts or news about the company."
)

analysis_task = Task(
    description="Perform a SWOT analysis of the company based on the research findings.",
    agent=analysis_agent,
    context=[research_task],
    expected_output="A SWOT analysis in four sections: Strengths, Weaknesses, Opportunities, and Threats."
)

writer_task = Task(
    description="Write an executive summary report of the company, using the SWOT analysis.",
    agent=writer_agent,
    context=[research_task, analysis_task],
    expected_output="A well-structured business report in natural language."
)

## 7. Run the Multi-Agent Crew
This runs the entire workflow: each agent receives the right context, and you get a business report.

In [3]:
crew = Crew(tasks=[research_task, analysis_task, writer_task])
result = crew.kickoff()
print(result)

**Executive Summary Report: Tesla, Inc. - Q3 2023 SWOT Analysis**

This report provides an analytical overview of Tesla, Inc. based on its Q3 2023 earnings report and a comprehensive SWOT analysis. Tesla has solidified its position as a leading force in the electric vehicle (EV) market while expanding its footprint in renewable energy, emphasizing the need to address challenges and leverage opportunities for future growth. 

**Strengths**  
1. **Dominant Market Position**: Tesla reported substantial vehicle deliveries, with over 435,000 units sold in Q3 2023 alone. This achievement underscores Tesla's leadership in the EV sector, highlighted by the Model Y becoming the best-selling vehicle in its class, thus surpassing traditional gasoline cars.  
   
2. **Strong Financial Performance**: The company realized an 18% year-over-year revenue increase, totaling $24.93 billion in Q3 2023. This financial growth reflects its solid health and enduring demand for its products.  
   
3. **Innovat